In [ ]:
# import the necessary packages
import numpy as np
import numpy.linalg as lin
import imutils
import cv2

class Stitcher:
	def __init__(self):
		# determine if we are using OpenCV v3.X
		self.isv3 = imutils.is_cv3()

    # If Point is in Square
	def isPinS(self, P, H, shape):
		TP = np.dot(lin.inv(H), [P, 1])
		TP = (D / D[2])[0:2]
		if TP[0]<shape[1]-1 and TP[0]>=0 and TP[1]<shape[0]-1 and TP[1]>=0:
			return True
		else:
			return False

    # w1~R : w1~PQ
	def weightrgb(self, w1, P, Q, R, Sc, TSc):
		weight = (w1-R[1])*(P[0]-Q[0])/((P[0]-R[0])*(w1-P[1])+(R[0]-Q[0])*(w1-Q[1]))
		return (Sc*weight + TSc*(1-weight),weight)

	def stitch(self, N, img, Center, ratio=0.75, reprojThresh=4.0, showMatches=False):
		# unpack the images, then detect keypoints and extract
		# local invariant descriptors from them
        features = [None]*N
        kps = [None]*N
        for i in range(0,N):
            (kps[i], features[i]) = self.detectAndDescribe(img[i])
            
		# match features between the two images
        M = [None]*(N-1)
        matches = [None]*(N-1)
        H = [None]*(N-1)
        status =[None]*(N-1)
        for i in range(0,N-1):
            M[i] = (self.matchKeypoints(kpsA, kpsB,featuresA, featuresB, ratio, reprojThresh))
            # if the match is None, then there aren't enough matched
            # keypoints to create a panorama
            if M is None:
                continue
            # otherwise, apply a perspective warp to stitch the images
            # together
            (matches[i], H[i], status[i]) = M
            
        # inverse Matrix of H
        invH = [None]*(N-1)
        for i in range(0,N):
            invH = np.linalg.inv(x)
            
        # Matrix from Center
        HC = [None]*N
        HC[Center] = np.eye(3)
        
        # Matrix Under Center
        TempH = np.eye(3)
        for i in range(Center-1,-1,-1):
            TempH = np.dot(TempH,invH[i])
            HC[i] = TempH
            
        # Matrix Over Center
        TempH = np.eye(3)
        for i in range(Center+1,N+1):
            TempH = np.dot(TempH,H[i-1])
            HC[i] = TempH
            
        # Parallel Transformation
        dis = (0, 0)
        for i in range(0,Center):
            dis[0] = dis[0] + img[i].shape[0]
            dis[1] = dis[1] + img[i].shape[1]
            
        PT = np.array([
            [1, 0, dis[0]],
            [0, 1, dis[1]],
            [0, 0, 1]
        ])
        
        # Parallel Transformed H
        PTHC = [None]*N
        PTHC = np.dot(PT,HC)
        
        imgSize = (0,0)
        for i in range(0,N):
            imgSize[0] += img[i].shape[0]
            imgSize[1] += img[i].shape[1]
        
        result = [None]*N
        for i in range(0,N):
            result[i] = cv2.warpPerspective(img[i]), PTHC[i], imgSize, borderMode=cv2.BORDER_CONSTANT, borderValue = BORDER.TRANSPARANT)
            cv2.imshow(result[i])
        return result
        
        
        '''
		result = cv2.warpPerspective(imageA, H,
			(imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
		
		newimageA= cv2.warpPerspective(imageA, H,
			(imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
		
		result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
		
		tmp = np.dot(H, [0, 0, 1])
		LU = (tmp/tmp[2])[0:2]
		tmp = np.dot(H, [imageA.shape[0]-1, 0, 1])
		LD = (tmp/tmp[2])[0:2]
		tmp = np.dot(H, [imageA.shape[0]-1, imageA.shape[1]-1, 1])
		RD = (tmp/tmp[2])[0:2]
		tmp = np.dot(H, [0, imageA.shape[1]-1, 1])
		RU = (tmp/tmp[2])[0:2]
		
		imageW = np.zeros((result.shape[0], result.shape[1]))
		for x in range(0, imageB.shape[0]):
			for y in range(0, imageB.shape[1]):
				if self.isinA(y, x, H, imageA):
					(r, weight) = self.weightrgb(imageB.shape[1]-1, LU[0], LU[1], RU[0], RU[1], y, x, imageB[x,y], newimageA[x,y])
					imageW[x,y] = weight
					result[x, y] = r
                                        
		cv2.imshow("newA", newimageA)
		cv2.imshow("W", imageW)
		# check to see if the keypoint matches should be visualized
		if showMatches:
			vis = self.drawMatches(imageA, imageB, kpsA, kpsB, matches,
				status)
        
			# return a tuple of the stitched image and the
			# visualization
			return (result, vis)
        '''
		# return the stitched image
		return result

	def detectAndDescribe(self, image):
		# convert the image to grayscale
		gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

		# check to see if we are using OpenCV 3.X
		if self.isv3:
			# detect and extract features from the image
			descriptor = cv2.xfeatures2d.SURF_create()
			(kps, features) = descriptor.detectAndCompute(image, None)

		# otherwise, we are using OpenCV 2.4.X
		else:
			# detect keypoints in the image
			# detector = cv2.FeatureDetector_create("SIFT")
			kps = detector.detect(gray)

			# extract features from the image
			# extractor = cv2.DescriptorExtractor_create("SIFT")
			(kps, features) = extractor.compute(gray, kps)

		# convert the keypoints from KeyPoint objects to NumPy
		# arrays
		kps = np.float32([kp.pt for kp in kps])

		# return a tuple of keypoints and features
		return (kps, features)

	def matchKeypoints(self, kpsA, kpsB, featuresA, featuresB,
		ratio, reprojThresh):
		# compute the raw matches and initialize the list of actual
		# matches
		matcher = cv2.DescriptorMatcher_create("FlannBased")
		rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
		matches = []

		# loop over the raw matches
		for m in rawMatches:
			# ensure the distance is within a certain ratio of each
			# other (i.e. Lowe's ratio test)
			if len(m) == 2 and m[0].distance < m[1].distance * ratio:
				matches.append((m[0].trainIdx, m[0].queryIdx))

		# computing a homography requires at least 4 matches
		if len(matches) > 4:
			# construct the two sets of points
			ptsA = np.float32([kpsA[i] for (_, i) in matches])
			ptsB = np.float32([kpsB[i] for (i, _) in matches])

			# compute the homography between the two sets of points
			(H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,
				reprojThresh)

			# return the matches along with the homograpy matrix
			# and status of each matched point
			return (matches, H, status)

		# otherwise, no homograpy could be computed
		return None

	def drawMatches(self, imageA, imageB, kpsA, kpsB, matches, status):
		# initialize the output visualization image
		(hA, wA) = imageA.shape[:2]
		(hB, wB) = imageB.shape[:2]
		vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
		vis[0:hA, 0:wA] = imageA
		vis[0:hB, wA:] = imageB

		# loop over the matches
		for ((trainIdx, queryIdx), s) in zip(matches, status):
			# only process the match if the keypoint was successfully
			# matched
			if s == 1:
				# draw the match
				ptA = (int(kpsA[queryIdx][0]), int(kpsA[queryIdx][1]))
				ptB = (int(kpsB[trainIdx][0]) + wA, int(kpsB[trainIdx][1]))
				cv2.line(vis, ptA, ptB, (0, 255, 0), 1)

		# return the visualization
		return vis
